<a href="https://colab.research.google.com/github/ffg-kom/mapping_evolution/blob/master/getting_data_ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

The following code was copied from https://colab.research.google.com/github/nestauk/im_tutorials/blob/master/notebooks/01_intro_to_pandas_tutorial.ipynb.

In [1]:
%load_ext autoreload
%autoreload 2

# install im_tutorial package
!pip install git+https://github.com/nestauk/im_tutorials.git

  Cloning https://github.com/nestauk/im_tutorials.git to /tmp/pip-req-build-6ywqz1d_
  Running command git clone -q https://github.com/nestauk/im_tutorials.git /tmp/pip-req-build-6ywqz1d_
  Created wheel for im-tutorials: filename=im_tutorials-0.1.1-cp36-none-any.whl size=15103 sha256=a9fd0f2d963585704d275bc2fee76e0a1c215a356c6ccf3ae8dce629c7e85e07
  Stored in directory: /tmp/pip-ephem-wheel-cache-wm33ww2a/wheels/47/a3/cb/bdc5f9ba49bcfd2c6864b166a1566eb2f104113bf0c3500330
Successfully built im-tutorials


In [0]:
# numpy for mathematical functions
import numpy as np
# pandas for handling tabular data
import pandas as pd
# explained later
from im_tutorials.data.cordis import h2020_projects

# for plotting
import matplotlib.pyplot as plt

# data
from im_tutorials.data.cordis import cordis_table

# Loading data

In [35]:
# load tables
cordis_o_df = cordis_table('organisations')
cordis_p_df = cordis_table('projects')
cordis_p_o_df = cordis_table('project_organisations')
cordis_pub_df = cordis_table('publications')

# merge them
cordis_project_orgs_df = cordis_p_o_df.merge(
  cordis_o_df, left_on='organization_id', right_on='id', how='left'
)
cordis_project_orgs_df = cordis_project_orgs_df.merge(
  cordis_p_df, left_on='project_rcn', right_on='rcn', how='left'
)

cordis_project_orgs_df.rename( columns={'contribution': 'ec_contribution_organization'
                                        , 'ec_contribution': 'ec_contribution_project'
                                        , 'total_cost': 'total_cost_project'
                                       }, inplace=True )

cordis_project_orgs_df.shape

(251040, 25)

In [36]:
# add number of partners per project as a column
count_partners_df = cordis_project_orgs_df[['project_rcn', 'organization_id']].groupby( by='project_rcn' ).count()
count_partners_df.rename( columns={'organization_id': 'number_of_partners'}, inplace=True )

cordis_project_orgs_df = cordis_project_orgs_df.merge(
  count_partners_df, on='project_rcn', how='left'
)

# add number of publications per project as a column
count_publications_df = cordis_pub_df[['id', 'project_rcn']].groupby( by='project_rcn' ).count()
count_publications_df.rename( columns={'id': 'number_of_publications'}, inplace=True )

cordis_project_orgs_df = cordis_project_orgs_df.merge(
  count_publications_df, on='project_rcn', how='left'
)


# shape: should by (something, 27) 
cordis_project_orgs_df.shape


(251040, 27)

In [0]:
# preparing dictionaries to effectively group data
dict_funding_schemes_along_programmes = {'BBI-CSA - Bio-based Industries Coordination and Support action': 'Bio-based Industries',
                                         'BBI-IA-DEMO - Bio-based Industries Innovation action - Demonstration': 'Bio-based Industries', 
                                         'BBI-IA-FLAG - Bio-based Industries Innovation action - Flagship': 'Bio-based Industries',
                                         'BBI-RIA - Bio-based Industries Research and Innovation action': 'Bio-based Industries', 
                                         'COFUND-EJP - COFUND (European Joint Programme)': '(general)', 
                                         'COFUND-PCP - COFUND (PCP)': '(general)', 
                                         'CS2-CSA - Coordination & support action': 'Clean Sky 2', 
                                         'CS2-IA - Innovation action': 'Clean Sky 2', 
                                         'CS2-RIA - Research and Innovation action': 'Clean Sky 2', 
                                         'CSA - Coordination and support action': '(general)', 
                                         'CSA-LS - CSA Lump sum': '(general)', 
                                         'ECSEL-CSA - ECSEL Coordination & Support action': 'ECSEL (Electronic Components and Systems for European Leadership)', 
                                         'ECSEL-IA - ECSEL Innovation Action': 'ECSEL (Electronic Components and Systems for European Leadership)', 
                                         'ECSEL-RIA - ECSEL Research and Innovation Action': 'ECSEL (Electronic Components and Systems for European Leadership)', 
                                         'ERA-NET-Cofund - ERA-NET Cofund': 'ERA-NET', 
                                         'ERC-ADG - Advanced Grant': 'ERC (European Research Council)', 
                                         'ERC-COG - Consolidator Grant': 'ERC (European Research Council)', 
                                         'ERC-POC - Proof of Concept Grant': 'ERC (European Research Council)', 
                                         'ERC-POC-LS - ERC Proof of Concept Lump Sum Pilot': 'ERC (European Research Council)', 
                                         'ERC-STG - Starting Grant': 'ERC (European Research Council)', 
                                         'FCH2-CSA - Coordination & support action': 'FCH2 (Fuel Cells and Hydrogen 2)', 
                                         'FCH2-IA - Innovation action': 'FCH2 (Fuel Cells and Hydrogen 2)', 
                                         'FCH2-RIA - Research and Innovation action': 'FCH2 (Fuel Cells and Hydrogen 2)', 
                                         'H2020-EEN-SGA - Specific Grant Agreement Enterprise Europe Network (EEN)': 'SME (Small and medium-sized Enterprises)', 
                                         'IA - Innovation action': '(general)', 
                                         'IA-LS - Innovation action Lump Sum': '(general)', 
                                         'IMI2-CSA - Coordination & support action': 'IMI2 (Innovative Medicines Initiative)', 
                                         'IMI2-RIA - Research and Innovation action': 'IMI2 (Innovative Medicines Initiative)', 
                                         'MSCA-COFUND-DP - Doctoral programmes': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'MSCA-COFUND-FP - Fellowship programmes': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'MSCA-IF-EF-CAR - CAR â€“ Career Restart panel': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'MSCA-IF-EF-RI - RI â€“ Reintegration panel': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'MSCA-IF-EF-SE - Society and Enterprise panel': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'MSCA-IF-EF-ST - Standard EF': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'MSCA-IF-GF - Global Fellowships': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'MSCA-ITN-EID - European Industrial Doctorates': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'MSCA-ITN-EJD - European Joint Doctorates': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'MSCA-ITN-ETN - European Training Networks': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'MSCA-RISE - Marie SkÅ‚odowska-Curie Research and Innovation Staff Exchange (RISE)': 'MSCA (Marie Skłodowska-Curie Actions)', 
                                         'PCP - Pre-Commercial Procurement': 'PCP & PPI (Pre-commercial or public Procurement)', 
                                         'PPI - Public Procurement of Innovative solutions': 'PCP & PPI (Pre-commercial or public Procurement)', 
                                         'RIA - Research and Innovation action': '(general)', 
                                         'RIA-LS - Research and Innovation action Lump Sum': '(general)', 
                                         'SESAR-CSA - Coordination and Support Action': 'SESAR (Single European Sky)', 
                                         'SESAR-IA - Innovation action': 'SESAR (Single European Sky)', 
                                         'SESAR-RIA - Research and Innovation action': 'SESAR (Single European Sky)', 
                                         'SGA-CSA - Specific Grant agreement and Coordination and Support Action': '(general)', 
                                         'SGA-RIA - SGA-RIA': '(general)', 
                                         'Shift2Rail-CSA - Coordination and Support Action': 'Shift2Rail', 
                                         'Shift2Rail-IA - Innovation action': 'Shift2Rail', 
                                         'Shift2Rail-IA-LS - Innovation Action Lump-Sum': 'Shift2Rail', 
                                         'Shift2Rail-RIA - Research and Innovation action': 'Shift2Rail', 
                                         'Shift2Rail-RIA-LS - Shift2Rail Research and Innovation Action Lump-Sum': 'Shift2Rail', 
                                         'SME-1 - SME instrument phase 1': 'SME (Small and medium-sized Enterprises)', 
                                         'SME-2 - SME instrument phase 2': 'SME (Small and medium-sized Enterprises)'
                                        }

dict_funding_schemes_along_actions = {'BBI-CSA - Bio-based Industries Coordination and Support action': 'CSA (Coordination and Support Action)',
                                      'BBI-IA-DEMO - Bio-based Industries Innovation action - Demonstration': 'IA (Innovation Action)',
                                      'BBI-IA-FLAG - Bio-based Industries Innovation action - Flagship': 'IA (Innovation Action)',
                                      'BBI-RIA - Bio-based Industries Research and Innovation action': 'RIA (Research and Innovation Action)',
                                      'COFUND-EJP - COFUND (European Joint Programme)': 'COFUND',
                                      'COFUND-PCP - COFUND (PCP)': 'COFUND',
                                      'CS2-CSA - Coordination & support action': 'CSA (Coordination and Support Action)',
                                      'CS2-IA - Innovation action': 'IA (Innovation Action)',
                                      'CS2-RIA - Research and Innovation action': 'RIA (Research and Innovation Action)',
                                      'CSA - Coordination and support action': 'CSA (Coordination and Support Action)',
                                      'CSA-LS - CSA Lump sum': 'CSA (Coordination and Support Action)',
                                      'ECSEL-CSA - ECSEL Coordination & Support action': 'CSA (Coordination and Support Action)',
                                      'ECSEL-IA - ECSEL Innovation Action': 'IA (Innovation Action)',
                                      'ECSEL-RIA - ECSEL Research and Innovation Action': 'RIA (Research and Innovation Action)',
                                      'ERA-NET-Cofund - ERA-NET Cofund': 'COFUND',
                                      'ERC-ADG - Advanced Grant': 'Grant',
                                      'ERC-COG - Consolidator Grant': 'Grant',
                                      'ERC-POC - Proof of Concept Grant': 'Grant',
                                      'ERC-POC-LS - ERC Proof of Concept Lump Sum Pilot': 'Grant',
                                      'ERC-STG - Starting Grant': 'Grant',
                                      'FCH2-CSA - Coordination & support action': 'CSA (Coordination and Support Action)',
                                      'FCH2-IA - Innovation action': 'IA (Innovation Action)',
                                      'FCH2-RIA - Research and Innovation action': 'RIA (Research and Innovation Action)',
                                      'H2020-EEN-SGA - Specific Grant Agreement Enterprise Europe Network (EEN)': 'SME (Small and medium-sized Enterprises)',
                                      'IA - Innovation action': 'IA (Innovation Action)',
                                      'IA-LS - Innovation action Lump Sum': 'IA (Innovation Action)',
                                      'IMI2-CSA - Coordination & support action': 'CSA (Coordination and Support Action)',
                                      'IMI2-RIA - Research and Innovation action': 'RIA (Research and Innovation Action)',
                                      'MSCA-COFUND-DP - Doctoral programmes': 'COFUND',
                                      'MSCA-COFUND-FP - Fellowship programmes': 'COFUND',
                                      'MSCA-IF-EF-CAR - CAR â€“ Career Restart panel': 'Fellowship',
                                      'MSCA-IF-EF-RI - RI â€“ Reintegration panel': 'Fellowship',
                                      'MSCA-IF-EF-SE - Society and Enterprise panel': 'Fellowship',
                                      'MSCA-IF-EF-ST - Standard EF': 'Fellowship',
                                      'MSCA-IF-GF - Global Fellowships': 'Fellowship',
                                      'MSCA-ITN-EID - European Industrial Doctorates': 'Innovative Training Networks',
                                      'MSCA-ITN-EJD - European Joint Doctorates': 'Innovative Training Networks',
                                      'MSCA-ITN-ETN - European Training Networks': 'Innovative Training Networks',
                                      'MSCA-RISE - Marie SkÅ‚odowska-Curie Research and Innovation Staff Exchange (RISE)': 'Research & Innovation Staff Exchange',
                                      'PCP - Pre-Commercial Procurement': 'Procurement',
                                      'PPI - Public Procurement of Innovative solutions': 'Procurement',
                                      'RIA - Research and Innovation action': 'RIA (Research and Innovation Action)',
                                      'RIA-LS - Research and Innovation action Lump Sum': 'RIA (Research and Innovation Action)',
                                      'SESAR-CSA - Coordination and Support Action': 'CSA (Coordination and Support Action)',
                                      'SESAR-IA - Innovation action': 'IA (Innovation Action)',
                                      'SESAR-RIA - Research and Innovation action': 'RIA (Research and Innovation Action)',
                                      'SGA-CSA - Specific Grant agreement and Coordination and Support Action': 'CSA (Coordination and Support Action)',
                                      'SGA-RIA - SGA-RIA': 'RIA (Research and Innovation Action)',
                                      'Shift2Rail-CSA - Coordination and Support Action': 'CSA (Coordination and Support Action)',
                                      'Shift2Rail-IA - Innovation action': 'IA (Innovation Action)',
                                      'Shift2Rail-IA-LS - Innovation Action Lump-Sum': 'IA (Innovation Action)',
                                      'Shift2Rail-RIA - Research and Innovation action': 'RIA (Research and Innovation Action)',
                                      'Shift2Rail-RIA-LS - Shift2Rail Research and Innovation Action Lump-Sum': 'RIA (Research and Innovation Action)',
                                      'SME-1 - SME instrument phase 1': 'SME (Small and medium-sized Enterprises)',
                                      'SME-2 - SME instrument phase 2': 'SME (Small and medium-sized Enterprises)'
                                     }


In [38]:
# add dictionaries as new columns
cordis_project_orgs_df['funding_scheme_programme'] = cordis_project_orgs_df['funding_scheme']
cordis_project_orgs_df['funding_scheme_programme'] = cordis_project_orgs_df['funding_scheme_programme'].map(dict_funding_schemes_along_programmes)

cordis_project_orgs_df['funding_scheme_action'] = cordis_project_orgs_df['funding_scheme']
cordis_project_orgs_df['funding_scheme_action'] = cordis_project_orgs_df['funding_scheme_action'].map(dict_funding_schemes_along_actions)


cordis_project_orgs_df.shape
    

(251040, 29)

In [39]:
# narrow our dataset down to what we want to work with
subset = cordis_project_orgs_df[
    (cordis_project_orgs_df.activity_type == 'Private for-profit entities (excluding Higher or Secondary Education Establishments)')
  & (cordis_project_orgs_df.framework == 'H2020')
]

df = subset[['organization_id', 'start_date_code', 'funding_scheme', 'funding_scheme_programme', 'funding_scheme_action', 'number_of_partners', 'number_of_publications', 'ec_contribution_organization', 'ec_contribution_project', 'total_cost_project']]
df.shape

(37138, 10)


# Playing with the data

(No need to to run anything from in here when preparing data for export.)

In [0]:
cordis_project_orgs_df.columns

Index(['project_rcn', 'organization_id', 'activity_type', 'address',
       'ec_contribution_organization', 'type', 'website_x', 'id', 'name',
       'country_code', 'country_name', 'rcn', 'acronym', 'end_date_code',
       'ec_contribution_project', 'framework', 'funding_scheme',
       'funded_under', 'objective', 'project_description', 'start_date_code',
       'status', 'title', 'total_cost_project', 'website_y',
       'number_of_partners'],
      dtype='object')

In [0]:
cordis_project_orgs_df['activity_type'].value_counts()

Higher or Secondary Education Establishments                                                         88335
Private for-profit entities (excluding Higher or Secondary Education Establishments)                 78810
Research Organisations                                                                               55099
Other                                                                                                15406
Public bodies (excluding Research Organisations and Secondary or Higher Education Establishments)    12454
                                                                                                       936
Name: activity_type, dtype: int64

In [0]:
# within subset, how often do organisations appear
subset['organization_id'].value_counts().head()



999993856    148
999960488     84
999909854     79
999951467     76
999908787     73
Name: organization_id, dtype: int64

In [0]:
subset[['organization_id', 'start_date_code', 'funding_scheme', 'number_of_partners', 'ec_contribution_organization', 'ec_contribution_project', 'total_cost_project']]


In [0]:
for i in range(100):
  print(cordis_project_orgs_df.loc[i + 10000].funded_under[0]['title'])



In [16]:
df['funding_scheme'].value_counts().to_csv('funding_scheme_grouping.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


# Preparing data for a Sankey diagram




In [40]:
# prepare the column that shall be used as the category shown in the sankey diagram
#   v   v   v   v   v   v   v   v   v   #
sankey_column = 'funding_scheme_action'
#   ^   ^   ^   ^   ^   ^   ^   ^   ^   #
df.rename(columns={sankey_column:'sankey'}, inplace=True)


# putting data in order
df = df.sort_values( by=['organization_id', 'start_date_code'] ).reset_index()

# add a column that numbers entries per organisation
df['organization_entry_number'] = df.groupby('organization_id').cumcount()

df[['organization_id', 'start_date_code', 'organization_entry_number']].head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,organization_id,start_date_code,organization_entry_number
0,888888310,2016-12-01,0
1,888888341,2018-01-01,0
2,888888620,2017-09-01,0
3,888888655,2017-03-01,0
4,888888707,2016-09-01,0


In [41]:
# looking at first two entries of organisations
first = df[df['organization_entry_number'] == 0]
second = df[df['organization_entry_number'] == 1]

onestep = second.merge(first, on='organization_id', how='left')

# aggregate over sankey-columns
sankey_df = onestep[['sankey_y', 'sankey_x', 'organization_id']].groupby(['sankey_y', 'sankey_x']).count()

# prepare for proper output (index -> columns, renaming)
sankey_df.reset_index(level=['sankey_y', 'sankey_x'], inplace=True)
sankey_df.rename(columns={'sankey_y':'source', 'sankey_x':'target'}, inplace=True)

# add a blank space to all targets (so targets differ from sources)
sankey_df['target'] = sankey_df['target'] + ' '

sankey_df.shape

(62, 3)

In [0]:
# export csv
sankey_df.to_csv('sankey_onestep_data.csv', index=False, sep=';')
